In [1]:
# --- Importaciones iniciales ---
import sys, os, importlib
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), "..", "scripts")))

import funciones_comunes
import funciones_mongo
import funciones_neo4j
import config_paths

importlib.reload(funciones_comunes)
importlib.reload(funciones_mongo)
importlib.reload(funciones_neo4j)

from funciones_comunes import cargar_csv
from funciones_neo4j import insertar_varias_relaciones
from funciones_neo4j import mostrar_relaciones



✅ Índices creados o ya existentes.
✅ Índices creados o ya existentes.


# Carga de datos de Mongo

In [2]:
# --- Limpiar colecciones antes de volver a insertar, las reinicia ---
funciones_mongo.db.usuarios.delete_many({})
funciones_mongo.db.destinos.delete_many({})
funciones_mongo.db.hoteles.delete_many({})
funciones_mongo.db.actividades.delete_many({})
funciones_mongo.db.reservas.delete_many({})

# --- Cargar datos desde JSON ---
usuarios = funciones_comunes.cargar_json(config_paths.USUARIOS_JSON)
destinos = funciones_comunes.cargar_json(config_paths.DESTINOS_JSON)
hoteles = funciones_comunes.cargar_json(config_paths.HOTELES_JSON)
actividades = funciones_comunes.cargar_json(config_paths.ACTIVIDADES_JSON)
reservas = funciones_comunes.cargar_json(config_paths.RESERVAS_JSON)

# --- Insertar en MongoDB ---
funciones_mongo.insertar_varios_documentos("usuarios", usuarios)
funciones_mongo.insertar_varios_documentos("destinos", destinos)
funciones_mongo.insertar_varios_documentos("hoteles", hoteles)
funciones_mongo.insertar_varios_documentos("actividades", actividades)
funciones_mongo.insertar_varios_documentos("reservas", reservas)


In [3]:
# --- Verificar que se insertaron correctamente ---
print("\n🧍 Usuarios:")
for elem in funciones_mongo.db.usuarios.find():
    print(elem)

print("\n🌍 Destinos:")
for elem in funciones_mongo.db.destinos.find():
    print(elem)

print("\n🏨 Hoteles:")
for elem in funciones_mongo.db.hoteles.find():
    print(elem)

print("\n🎯 Actividades:")
for elem in funciones_mongo.db.actividades.find():
    print(elem)

print("\n📅 Reservas:")
for elem in funciones_mongo.db.reservas.find():
    print(elem)


🧍 Usuarios:
{'_id': ObjectId('68f9b451a949eb4e12dfaf99'), 'usuario_id': 1, 'nombre': 'María Pérez', 'email': 'maria.perez@example.com', 'telefono': '+54 11 4567 1234'}
{'_id': ObjectId('68f9b451a949eb4e12dfaf9a'), 'usuario_id': 2, 'nombre': 'Juan López', 'email': 'juan.lopez@example.com', 'telefono': '+54 221 334 5566'}
{'_id': ObjectId('68f9b451a949eb4e12dfaf9b'), 'usuario_id': 3, 'nombre': 'Carla Gómez', 'email': 'carla.gomez@example.com', 'telefono': '+54 261 789 2233'}
{'_id': ObjectId('68f9b451a949eb4e12dfaf9c'), 'usuario_id': 4, 'nombre': 'Luis Fernández', 'email': 'luis.fernandez@example.com', 'telefono': '+54 299 444 9988'}
{'_id': ObjectId('68f9b451a949eb4e12dfaf9d'), 'usuario_id': 5, 'nombre': 'Ana Torres', 'email': 'ana.torres@example.com', 'telefono': '+54 381 123 4567'}

🌍 Destinos:
{'_id': ObjectId('68f9b451a949eb4e12dfaf9e'), 'destino_id': 1, 'ciudad': 'Bariloche', 'pais': 'Argentina', 'tipo': 'Montaña', 'precio_promedio': 90000}
{'_id': ObjectId('68f9b451a949eb4e12dfaf

# NEO4J

In [4]:
from funciones_neo4j import insertar_varios_nodos, insertar_varias_relaciones, mostrar_relaciones
import funciones_mongo
from funciones_comunes import cargar_csv  # si la usas para leer CSV

# === NODOS ===
usuarios = funciones_mongo.db.usuarios.find()
usuarios_formateados = [
    {"usuario_id": u["usuario_id"], "nombre": u["nombre"], "email": u["email"], "telefono": u["telefono"]}
    for u in usuarios
]
insertar_varios_nodos("Usuario", usuarios_formateados)

destinos = funciones_mongo.db.destinos.find()
destinos_formateados = [
    {"destino_id": d["destino_id"], "ciudad": d["ciudad"], "pais": d["pais"], "tipo": d["tipo"], "precio_promedio": d["precio_promedio"]}
    for d in destinos
]
insertar_varios_nodos("Destino", destinos_formateados)

# === RELACIONES ===
relaciones_sociales = cargar_csv("relaciones_sociales.csv")
rel_social_formateadas = [
    {
        "label_origen": "Usuario",
        "prop_origen": "usuario_id",
        "valor_origen": int(r["usuario1_id"]),
        "label_destino": "Usuario",
        "prop_destino": "usuario_id",
        "valor_destino": int(r["usuario2_id"]),
        "tipo": r["tipo"]
    }
    for r in relaciones_sociales
]

visitas = cargar_csv("visitas.csv")
rel_visitas_formateadas = [
    {
        "label_origen": "Usuario",
        "prop_origen": "usuario_id",
        "valor_origen": int(v["usuario_id"]),
        "label_destino": "Destino",
        "prop_destino": "destino_id",
        "valor_destino": int(v["destino_id"]),
        "tipo": "VISITO"
    }
    for v in visitas
]

# === INSERTAR RELACIONES ===
insertar_varias_relaciones(rel_social_formateadas)
insertar_varias_relaciones(rel_visitas_formateadas)

# === MOSTRAR RELACIONES ===
mostrar_relaciones()

✅ Se insertaron 5 nodos en 'Usuario'.
✅ Se insertaron 5 nodos en 'Destino'.
✅ Se insertaron 3 relaciones.
✅ Se insertaron 7 relaciones.

🔹 Relaciones sociales (Usuario ↔ Usuario):
  1 -[AMIGO_DE]-> 2
  2 -[AMIGO_DE]-> 1
  3 -[FAMILIAR_DE]-> 4

🔹 Relaciones de visitas (Usuario → Destino):
  1 -[VISITO]-> 1
  1 -[VISITO]-> 5
  2 -[VISITO]-> 1
  3 -[VISITO]-> 3
  4 -[VISITO]-> 2
  5 -[VISITO]-> 1
  5 -[VISITO]-> 4
